In [ ]:
from transformers import  AutoModel,AutoTokenizer,OPTForCausalLM
import os
output_dir = '/workspace/llama2/dpo-gsm8k-1e5-beta-0.01-ep1'
from datasets import load_dataset
data= load_dataset('/workspace/ye/dataset/processed_Dahoas/full_hh_rlhf')
class Simple_Generate():
    def __init__(self,model_path):
        self.model= OPTForCausalLM.from_pretrained(model_path,trust_remote_code=True).half().to('cuda:2')
        self.tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)
        self.tokenizer.pad_token = self.tokenizer.eos_token
    def forward(self,prompt):
        inputs = self.tokenizer.batch_encode_plus([prompt], return_tensors="pt",padding=True).to('cuda:2')
        # print(inputs)
        outputs = self.model.generate(**inputs,do_sample=True,max_new_tokens=1024,eos_token_id=2)
        # print(len(outputs[0].tolist()))
        # print(outputs[0].tolist())
        response = self.tokenizer.decode(outputs[0].tolist()[len(inputs['input_ids'][0]):])
        print(response)
        return response
Simple_Generate = Simple_Generate(os.path.join(output_dir))

In [ ]:
tokenizer.add_special_tokens({'additional_special_tokens': ['<endoftext>']})

In [ ]:
result = []
for i in range(20):
    result.append(Simple_Generate.forward(data['train']['prompt'][i]))
    print("-----------------------------------------------------------------------")

In [ ]:
print(result[6])

In [1]:
from transformers import  AutoModel,AutoTokenizer,OPTForCausalLM
import os
output_dir = '/workspace/ye/DeepSpeed-Chat/training/step3_rlhf_finetuning/output/actor'
from datasets import load_dataset
data= load_dataset('/workspace/ye/dataset/processed_Dahoas/full_hh_rlhf')
class Simple_Generate():
    def __init__(self,model_path):
        self.model= OPTForCausalLM.from_pretrained(model_path,trust_remote_code=True).half().to('cuda:5')
        self.tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)
        self.tokenizer.pad_token = self.tokenizer.eos_token
    def forward(self,prompt):
        inputs = self.tokenizer.batch_encode_plus([prompt], return_tensors="pt",padding=True).to('cuda:5')
        # print(inputs)
        outputs = self.model.generate(**inputs,do_sample=True,max_new_tokens=512,eos_token_id=50260)
        # print(len(outputs[0].tolist()))
        # print(outputs[0].tolist())
        response = self.tokenizer.decode(outputs[0].tolist()[len(inputs['input_ids'][0]):])
        print(response)
        return response
Simple_Generate2 = Simple_Generate(os.path.join(output_dir))

/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Generating train split: 112052 examples [00:00, 259273.36 examples/s]
Generating test split: 12451 examples [00:00, 243398.22 examples/s]
The argument `trust_remote_code` is to be used with Auto classes. It has no effect here and is ignored.


In [5]:
data['train']

Dataset({
    features: ['prompt', 'response', 'chosen', 'rejected'],
    num_rows: 112052
})

In [9]:
data['train']['prompt'][1]

"\n\nHuman: I'm trying to learn about the salam witch trials\n\nAssistant: If you’re looking for books about witchcraft trials, I can recommend some books for you. But you probably mean you’re looking for more historical information about the Salem witch trials in 1692, and specifically about the salam witch trials in 1692?<|endoftext|>\n\nHuman: What are some good books to learn about the salam witch trials\n\nAssistant:"

In [2]:
result = []
for i in range(20):
    result.append(Simple_Generate2.forward(data['train']['prompt'][i]))
    print("-----------------------------------------------------------------------")

 You’re welcome.  That was what I was there to answer.  :)  I hope you have a great day!<|endoftext|>
-----------------------------------------------------------------------
 Here are some books about the Salem witch trials:

The Pilgrims in Salem witch trial: the making of a religious war

A witch hunt in upstate New York 1692

The Salem Witch Trials Book by Margaret Stoecker; the first great non-fiction book about the witch trial trials in Salem, Massachusetts

The Salem Witch Trials: a History of the Witch Hunt

Witch Trial: The Greatest American Story Never Told

Salem Witch Trials: A History From the Perspective of Women

The Witch Trial: The Greatest American Story Ever Told

Salem Witch Trials: The Greatest American Story Ever Told (2nd ed.)

From Salem witch trial to the war on terror: the story of the Salem witch trials in America, 1820 to the present day

Salem Witch Trials: The Greatest American Story Ever Told (3rd ed.)

The Purge: A Report by the Department of Justice

Wit

In [ ]:
result

In [ ]:
from datasets import load_dataset
import pandas as pd
data= load_dataset('/workspace/ye/dataset/processed_Dahoas/rm-static')
def func(data):
    data['prompt'] = '\n\nHuman:'+"<|endoftext|>\n\nHuman:".join(data['prompt'].split('\n\nHuman:')[1:])
    data['chosen'] = data['chosen']
    data['rejected'] = data['rejected']
    return data
data['train'] = data['train'].map(func)
data['test'] = data['test'].map(func)

train_df = pd.DataFrame(data['train'])
test_df = pd.DataFrame(data['test'])

train_df.to_parquet('/workspace/ye/dataset/yitingxie/processed-rlhf-reward-datasets/train.parquet')
test_df.to_parquet('/workspace/ye/dataset/yitingxie/processed-rlhf-reward-datasets/test.parquet')

In [ ]:
from transformers import  AutoModel,AutoTokenizer,OPTForCausalLM
t = AutoTokenizer.from_pretrained('/workspace/ye/DeepSpeed-Chat/training/step1_supervised_finetuning/output')